In [166]:
import pandas as pd
# import textdistance as td

In [167]:
xls = pd.ExcelFile('data.xlsx')

df_emis = pd.read_excel(xls, 'EMIS')
df_payroll = pd.read_excel(xls, 'Payroll')

df_emis.head()

,Numéro EMIS,teacher_surname,teacher_name,teacher_sex,Date of birth
0,1,2Nrydoss,Guvyugu,Female,1978-01-16
1,2,Adakma - Bahy,Itku W,Female,1988-04-12
2,3,Adakma-Gdumm,Axydi,Female,1989-12-29
3,4,Adoyhhi,Gaziz,Male,1964-07-29
4,5,Adoyhhi,Gaziz,Male,1984-10-06


In [168]:
df_payroll.head()

,Numéro Solde,Surname,First name,Date of birth
0,1,A’Koyh,Urkiz,1961-02-15
1,2,Adakma-Gdumm,Axydi Ixyfupinl,1989-12-29
2,3,Adoyhhi,Wvykqiz Uereznu,1961-11-15
3,4,Aeykxyka,Oagpu,1967-04-24
4,5,Afykn-Punak,Utiaxu,1999-04-24


In [173]:
df_emis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29875 entries, 0 to 29874
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Numéro EMIS      29875 non-null  int64         
 1   teacher_surname  29875 non-null  object        
 2   teacher_name     29875 non-null  object        
 3   teacher_sex      29875 non-null  object        
 4   Date of birth    29875 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.1+ MB


In [174]:
df_payroll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29526 entries, 0 to 29525
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Numéro Solde   29526 non-null  int64         
 1   Surname        29526 non-null  object        
 2   First name     29526 non-null  object        
 3   Date of birth  29526 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 922.8+ KB


In [175]:
from string import digits
import re

class Utils:

    def __init__(self):
        self.filter = lambda x: self.filter_string(x)

    def rename_columns(self, df, old, new):

        dict = {}

        for i, column in enumerate(old):
            dict[column] = new[i]

        df = df.rename(columns=dict)

        return df

    def filter_string(self, string):
        # lower case
        string = string.lower()

        # remove digits
        remove_digits = str.maketrans('', '', digits)
        string = string.translate(remove_digits)

        # remove punctuations
        # string = re.sub(r'[^\w\s]', '', string)

        # # remove white spaces
        # string = string.replace(" ", "")
        # string = string.replace("  ", "")
        return string

    def filter_df(self, df):

        df['first_name'] = df['first_name'].apply(self.filter)
        df['last_name'] = df['last_name'].apply(self.filter)

        return df

In [176]:
utils = Utils()

emis = utils.rename_columns(df_emis, df_emis.columns, ['id', 'last_name', 'first_name', 'sex', 'dob'])
payroll = utils.rename_columns(df_payroll, df_payroll.columns, ['id', 'last_name', 'first_name', 'dob'])

emis = utils.filter_df(emis)
payroll = utils.filter_df(payroll)

In [177]:
from math import floor

class SimilarityMeasurer:

    def __init__(self):
        pass

    def get_similarity(self, d1, d2):
        fname_sim_score = self._jaro_winkler(d1[2], d2[2])
        lname_sim_score = self._jaro_winkler(d1[1], d2[1])
        return [fname_sim_score, lname_sim_score]
    
    def get_date_score(self, day1, day2):
      date_score = 0
      if day1 == day2:
        date_score = 1
      elif day1 == 1 or day2 == 1:
        date_score = 0.5
      
      return date_score
    
    def _jaro_distance(self, s1, s2):

        # If the strings are equal
        if s1 == s2:
            return 1.0

        # Length of two strings
        len_s1 = len(s1)
        len_s2 = len(s2)

        if (len_s1 == 0 or len_s2 == 0):
            return 0.0

            # Maximum distance upto which matching is allowed
        max_dist = (max(len_s1, len_s2) // 2) - 1

        # Count of matches
        match = 0

        # Hash for matches
        hash_s1 = [0] * len_s1
        hash_s2 = [0] * len_s2

        # Traverse through the first string
        for i in range(len_s1):
            for j in range(max(0, i - max_dist), min(len_s2, i + max_dist + 1)):
                # If there is a match
                if (s1[i] == s2[j] and hash_s2[j] == 0):
                    hash_s1[i] = 1
                    hash_s2[j] = 1
                    match += 1
                    break

                    # If there is no match
        if (match == 0):
            return 0.0;

        # Number of transpositions
        t = 0
        point = 0

        # Count number of occurances where two characters match but
        # there is a third matched character in between the indices

        for i in range(len_s1):
            if (hash_s1[i]):

                # Find the next matched character in second string
                while (hash_s2[point] == 0):
                    point += 1

                if (s1[i] != s2[point]):
                    point += 1
                    t += 1
                else:
                    point += 1

            t /= 2

            # Return the Jaro Similarity
        return ((match / len_s1 + match / len_s2 +
                 (match - t) / match) / 3.0)

    def _jaro_winkler(self, s1, s2):

        jaro_dist = self._jaro_distance(s1, s2)

        # If the jaro Similarity is above a threshold
        if (jaro_dist > 0.7):

            # Find the length of common prefix
            prefix = 0

            for i in range(min(len(s1), len(s2))):
                # If the characters match
                if (s1[i] == s2[i]):
                    prefix += 1
                else:
                    break

            # Maximum of 4 characters are allowed in prefix
            prefix = min(4, prefix)

            # Calculate jaro winkler Similarity
            jaro_dist += 0.1 * prefix * (1 - jaro_dist)

        return jaro_dist

In [236]:
from time import gmtime, strftime

sm = SimilarityMeasurer()

exact_data = []
ghost_data = []
strong_data = []
medium_data = []
low_data = []

print(strftime("%H:%M:%S", gmtime()))

for each_emis in emis.values[0:1000]:
    
    # Limit search space by searching only among records with first names starting with first 2 initials same as that of record
    sub_payroll = payroll[payroll['first_name'].str.startswith(each_emis[2][0:2])]

    for each_pr in sub_payroll.values:
        score_fname, score_lname = sm.get_similarity(each_emis, each_pr)
        score = round((score_fname + score_lname) / 2, 2)
        
        if each_emis[4] == each_pr[3]:
          if score_fname > 0.85 and score_lname > 0.85:
            ex_data = {
                'id': each_emis[0],
                'last_name_emis': df_emis.loc[df_emis['Numéro EMIS'] == each_emis[0]].values[0][1],
                'first_name_emis': df_emis.loc[df_emis['Numéro EMIS'] == each_emis[0]].values[0][2],
                'sex_emis': each_emis[3],
                'dob_emis': each_emis[4],
                'match': each_pr[0],
                'last_name_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][1],
                'first_name_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][2],
                'dob_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][3],
              }
            exact_data.append(ex_data)
        elif each_emis[4].year == each_pr[3].year and each_emis[4].month == each_pr[3].month:
          if score_fname > 0.85 and score_lname > 0.85:
            st_data = {
              'id': each_emis[0],
              'last_name_emis': df_emis.loc[df_emis['Numéro EMIS'] == each_emis[0]].values[0][1],
              'first_name_emis': df_emis.loc[df_emis['Numéro EMIS'] == each_emis[0]].values[0][2],
              'sex_emis': each_emis[3],
              'dob_emis': each_emis[4],
              'match': each_pr[0],
              'last_name_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][1],
              'first_name_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][2],
              'dob_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][3],
              'score': str(score * 100) + '%'
            }
            strong_data.append(st_data)
          elif score_fname < 0.5 and score_lname < 0.5:
            lw_data = {
              'id': each_emis[0],
              'last_name_emis': df_emis.loc[df_emis['Numéro EMIS'] == each_emis[0]].values[0][1],
              'first_name_emis': df_emis.loc[df_emis['Numéro EMIS'] == each_emis[0]].values[0][2],
              'sex_emis': each_emis[3],
              'dob_emis': each_emis[4],
              'match': each_pr[0],
              'last_name_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][1],
              'first_name_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][2],
              'dob_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][3],
              'score': str(score * 100) + '%'
            }
            low_data.append(lw_data)
          else:
            md_data = {
              'id': each_emis[0],
              'last_name_emis': df_emis.loc[df_emis['Numéro EMIS'] == each_emis[0]].values[0][1],
              'first_name_emis': df_emis.loc[df_emis['Numéro EMIS'] == each_emis[0]].values[0][2],
              'sex_emis': each_emis[3],
              'dob_emis': each_emis[4],
              'match': each_pr[0],
              'last_name_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][1],
              'first_name_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][2],
              'dob_pr': df_payroll.loc[df_payroll['Numéro Solde'] == each_pr[0]].values[0][3],
              'score': str(score * 100) + '%'
            }
            medium_data.append(md_data)

df_exact = pd.DataFrame(columns=['id','last_name_emis','first_name_emis','sex_emis','dob_emis','match','last_name_pr','first_name_pr','dob_pr'], data=exact_data)
df_ghost = pd.DataFrame(columns=['id','last_name_emis','first_name_emis','sex_emis','dob_emis','match', 'last_name_pr','first_name_pr','dob_pr'])
df_strong = pd.DataFrame(columns=['id','last_name_emis','first_name_emis','sex_emis','dob_emis','match','last_name_pr','first_name_pr','dob_pr','score'], data=strong_data)
df_medium = pd.DataFrame(columns=['id','last_name_emis','first_name_emis','sex_emis','dob_emis','match','last_name_pr','first_name_pr','dob_pr','score'], data=medium_data)
df_low = pd.DataFrame(columns=['id','last_name_emis','first_name_emis','sex_emis','dob_emis','match','last_name_pr','first_name_pr','dob_pr','score'], data=low_data)

print(strftime("%H:%M:%S", gmtime()))

14:54:48
14:55:30


In [237]:
df_exact.head()

,id,last_name_emis,first_name_emis,sex_emis,dob_emis,match,last_name_pr,first_name_pr,dob_pr
0,3,Adakma-Gdumm,Axydi,Female,1989-12-29,2,Adakma-Gdumm,Axydi Ixyfupinl,1989-12-29
1,6,Adoyhhi,Wvykqizz Uereznu,Female,1961-11-15,3,Adoyhhi,Wvykqiz Uereznu,1961-11-15
2,7,Aeykxyka,Oagpu,Male,1967-04-24,4,Aeykxyka,Oagpu,1967-04-24
3,8,Afykn-Punak,Utiaxu,Male,1999-04-24,5,Afykn-Punak,Utiaxu,1999-04-24
4,10,Ahu Fohhoupn,Ixyfipinl,Female,1964-10-09,2008,Fohhoupn,Ixyfupinl Axu,1964-10-09


In [238]:
df_low.head()

,id,last_name_emis,first_name_emis,sex_emis,dob_emis,match,last_name_pr,first_name_pr,dob_pr,score
0,43,Baapjyd,Galuggit,Male,1987-11-05,10041,Nukkar,Gavyzzak Sekayv,1987-11-18,45.0%
1,423,Bakmyr,Gupykno Geggo,Female,1965-07-29,19172,Quhhak,Guzzul,1965-07-03,25.0%
2,424,Bakmyr,Gupykno S.,Female,1999-05-29,10591,Nuoxe,Gulgaet,1999-05-26,25.0%
3,440,Bakmyr,Guvo Salk,Female,1988-09-24,6286,Jur,Gujuxxuo,1988-09-02,25.0%
4,448,Bakmyr,Guvruvinni,Female,1993-11-16,21488,Qupudu,Gugaet Gezu,1993-11-17,23.0%


In [239]:
df_strong.head()

,id,last_name_emis,first_name_emis,sex_emis,dob_emis,match,last_name_pr,first_name_pr,dob_pr,score
0,141,Bakkyr,Galuggit. Zus.,Male,1960-02-06,325,Bakmyr,Galugit Z.,1960-02-13,93.0%
1,147,Bakmykr,Uptexyu Z,Male,1994-03-13,891,Bakmyr,Uptex J.,1994-03-20,93.0%
2,159,Bakmyr,Azguk,Male,1997-11-12,157,Bakmyr,Azguk .U,1997-11-03,96.0%
3,162,Bakmyr,Azguk Z,Male,1960-07-26,141,Bakmyr,Azguk,1960-07-21,96.0%
4,167,Bakmyr,Azguk U,Male,1997-11-03,148,Bakmyr,Azguk,1997-11-12,97.0%


In [240]:
df_medium.head()

,id,last_name_emis,first_name_emis,sex_emis,dob_emis,match,last_name_pr,first_name_pr,dob_pr,score
0,1,2Nrydoss,Guvyugu,Female,1978-01-16,11246,Nyhhe,Gurtuxiki,1978-01-09,56.99999999999999%
1,1,2Nrydoss,Guvyugu,Female,1978-01-16,12903,Nyyhui,Guou,1978-01-22,66.0%
2,1,2Nrydoss,Guvyugu,Female,1978-01-16,13563,Parupyx,Gurtuxiki,1978-01-14,51.0%
3,1,2Nrydoss,Guvyugu,Female,1978-01-16,14640,Puknudui,Guvyugu,1978-01-02,76.0%
4,1,2Nrydoss,Guvyugu,Female,1978-01-16,19829,Quke,Guvyune,1978-01-03,44.0%


In [241]:
df_exact.to_excel('exact.xls')
df_strong.to_excel('strong.xls')
df_medium.to_excel('medium.xls')
df_low.to_excel('low.xls')